In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import json
import datetime

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
dataset_name = "SEG_CNNLSTM_2_retrain"

In [4]:
hyper_tuning_version = "ht_1"

In [5]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
if not os.path.exists("version"):
    os.makedirs("version")
version_dir = "version/" + timestamp 

timestamp

'20201203-191544'

vocabulary = np.genfromtxt("static/vocabulary.csv", delimiter="\n", dtype=np.int64)
vocab_size = vocabulary.shape[0]
vocab_size

with open("best_hps/{}/best_hp_{}.json".format(hyper_tuning_version, 0), "r") as j:
    print(json.load(j))

In [6]:
param_list = dict()

param_list["PAST_HISTORY"] = 16
param_list["FUTURE_TARGET"] = 8
param_list["BATCH_SIZE"] = 128
param_list["EPOCHS"] = 1000
param_list["BUFFER_SIZE"] = 200000
param_list["VOCAB_SIZE"] = 16293        #vocab_size

param_list['FILTERS'] = 208
param_list['KERNEL_SIZE'] = 8
param_list['NUM_1_NEURONS'] = 112
param_list['DROPOUT_1'] = 0.35
param_list['NUM_2_NEURONS'] = 112
param_list['DROPOUT_2'] = 0.35
#param_list['LEARNING_RATE'] = 0.001

In [7]:
train_set = np.genfromtxt("data/{}_train_set.csv".format(dataset_name), delimiter="\n", dtype=np.int64)

In [8]:
x_train = tf.data.Dataset.from_tensor_slices(train_set[:-param_list["FUTURE_TARGET"]]).window(param_list["PAST_HISTORY"], 1, 1, True)
# As dataset.window() returns "dataset", not "tensor", need to flat_map() it with sequence length
x_train = x_train.flat_map(lambda x: x.batch(param_list["PAST_HISTORY"])) 
x_train = x_train.map(lambda x: tf.one_hot(x, param_list["VOCAB_SIZE"], axis=-1))
x_train = x_train.batch(param_list["BATCH_SIZE"])

In [9]:
y_train = tf.data.Dataset.from_tensor_slices(train_set[param_list["PAST_HISTORY"]:]).window(param_list["FUTURE_TARGET"], 1, 1, True)
y_train = y_train.flat_map(lambda y: y.batch(param_list["FUTURE_TARGET"]))
y_train = y_train.map(lambda y: tf.one_hot(y, param_list["VOCAB_SIZE"], axis=-1))
y_train = y_train.batch(param_list["BATCH_SIZE"])

In [10]:
train_data = tf.data.Dataset.zip((x_train, y_train))

In [11]:
val_set = np.genfromtxt("data/{}_val_set.csv".format(dataset_name), delimiter="\n", dtype=np.int64)

In [12]:
x_val = tf.data.Dataset.from_tensor_slices(val_set[:-param_list["FUTURE_TARGET"]]).window(param_list["PAST_HISTORY"], 1, 1, True)
x_val = x_val.flat_map(lambda x: x.batch(param_list["PAST_HISTORY"]))
x_val = x_val.map(lambda x: tf.one_hot(x, param_list["VOCAB_SIZE"], axis=-1))
x_val = x_val.batch(param_list["BATCH_SIZE"])

In [13]:
y_val = tf.data.Dataset.from_tensor_slices(val_set[param_list["PAST_HISTORY"]:]).window(param_list["FUTURE_TARGET"], 1, 1, True)
y_val = y_val.flat_map(lambda y: y.batch(param_list["FUTURE_TARGET"]))
y_val = y_val.map(lambda y: tf.one_hot(y, param_list["VOCAB_SIZE"], axis=-1))
y_val = y_val.batch(param_list["BATCH_SIZE"])

In [14]:
val_data = tf.data.Dataset.zip((x_val, y_val))

In [15]:
model = keras.models.Sequential()
model.add(keras.layers.Conv1D(
    filters=param_list["FILTERS"], 
    kernel_size=param_list["KERNEL_SIZE"],
    padding="causal", activation='relu'))
model.add(keras.layers.MaxPooling1D(
    pool_size=param_list["PAST_HISTORY"]//param_list["FUTURE_TARGET"]       # Must be Static
    ))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(
    param_list["NUM_1_NEURONS"],
    return_sequences=True)))
model.add(keras.layers.Dropout(
    param_list["DROPOUT_1"]
    ))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(
    param_list["NUM_2_NEURONS"],
    return_sequences=True)))
model.add(keras.layers.Dropout(
    param_list["DROPOUT_2"]
    ))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(param_list["VOCAB_SIZE"], activation="softmax")))

model.compile(optimizer=keras.optimizers.Nadam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model_history = model.fit(train_data, epochs=param_list["EPOCHS"], validation_data=val_data, callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=5)])

Epoch 1/1000
     39/Unknown - 5s 121ms/step - loss: 3.6473 - accuracy: 0.8270

KeyboardInterrupt: 

In [17]:
model.save(version_dir)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: version/20201123-091640\assets


In [ ]:
# Version Issue
model.save(version_dir + "/model.h5")